In [10]:
import os, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import pennylane as qml

print("CUDA_VISIBLE_DEVICES =", os.environ.get("CUDA_VISIBLE_DEVICES"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("PyTorch device:", device)

torch.set_default_dtype(torch.float64)
np.set_printoptions(precision=4, suppress=True)

CUDA_VISIBLE_DEVICES = None
PyTorch device: cuda


In [11]:
CSV_PATH = "QIC2025-EstDat.csv"
if not os.path.exists(CSV_PATH):
    alt = "QIC2025-EstDat (1).csv"
    if os.path.exists(alt):
        CSV_PATH = alt

df = pd.read_csv(CSV_PATH)

TARGET   = "DV"
FEATURES = ["BW","COMED","DOSE","TIME","EVID","MDV","AMT","CMT","DVID"]
df = df.dropna(subset=FEATURES + [TARGET]).copy()

print("Satır:", len(df), "| Özellik sayısı:", len(FEATURES))
df.head(3)

Satır: 2820 | Özellik sayısı: 9


,ID,BW,COMED,DOSE,TIME,DV,EVID,MDV,AMT,CMT,DVID
0,1,58,0,0,0,18.6174,0,0,0,3,2
1,1,58,0,0,1,13.7783,0,0,0,3,2
2,1,58,0,0,2,16.5747,0,0,0,3,2


In [12]:
rng = np.random.default_rng(42)
uids = df["ID"].unique().copy()
rng.shuffle(uids)

n_total = len(uids)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)

train_ids = uids[:n_train]
val_ids   = uids[n_train:n_train+n_val]
test_ids  = uids[n_train+n_val:]

train_df = df[df["ID"].isin(train_ids)].copy()
val_df   = df[df["ID"].isin(val_ids)].copy()
test_df  = df[df["ID"].isin(test_ids)].copy()


print("Train/Val/Test şekilleri:", train_df.shape, val_df.shape, test_df.shape)

Train/Val/Test şekilleri: (2040, 11) (355, 11) (425, 11)


In [13]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
import numpy as np

X_train_raw = train_df[FEATURES].to_numpy(dtype=np.float64)
y_train_raw = train_df[TARGET].to_numpy(dtype=np.float64)

scaler_mi = StandardScaler().fit(X_train_raw)
X_train_std = scaler_mi.transform(X_train_raw)

mi = mutual_info_regression(X_train_std, y_train_raw, random_state=42)
top_idx = np.argsort(mi)[::-1][:3]
TOP3_FEATURES = [FEATURES[i] for i in top_idx]

print("MI skorları:", dict(zip(FEATURES, mi)))
print("Seçilen TOP3:", TOP3_FEATURES)

MI skorları: {'BW': 0.3037416872147447, 'COMED': 0.04074290033175565, 'DOSE': 0.23695951682639427, 'TIME': 0.4901808746020251, 'EVID': 0.592286523888653, 'MDV': 0.5807141068924297, 'AMT': 0.59223657912797, 'CMT': 0.7974154220004936, 'DVID': 0.8059427019699386}
Seçilen TOP3: ['DVID', 'CMT', 'EVID']


In [14]:
FEATURES_USED = TOP3_FEATURES
X_train = train_df[FEATURES_USED].to_numpy(dtype=np.float64)
X_val   = val_df[FEATURES_USED].to_numpy(dtype=np.float64)
X_test  = test_df[FEATURES_USED].to_numpy(dtype=np.float64)

y_train = train_df[TARGET].to_numpy(dtype=np.float64)
y_val   = val_df[TARGET].to_numpy(dtype=np.float64)
y_test  = test_df[TARGET].to_numpy(dtype=np.float64)

mean_ = X_train.mean(axis=0, keepdims=True)
std_  = X_train.std(axis=0, keepdims=True)
std_[std_ == 0] = 1.0
eps = 1e-8

def zscore(x): return (x - mean_) / (std_ + eps)

TrainX = zscore(X_train).astype(np.float64)  # (N, 3)
ValX   = zscore(X_val).astype(np.float64)    # (N, 3)
TestX  = zscore(X_test).astype(np.float64)   # (N, 3)

print("Shapes (top-3, no pad):", TrainX.shape, ValX.shape, TestX.shape)  # (..., 3)

Shapes (top-3, no pad): (2040, 3) (355, 3) (425, 3)


In [17]:
Xtr = torch.from_numpy(TrainX)                
Ytr = torch.from_numpy(y_train.reshape(-1,1))
Xva = torch.from_numpy(ValX)
Yva = torch.from_numpy(y_val.reshape(-1,1))
Xte = torch.from_numpy(TestX)
Yte = torch.from_numpy(y_test.reshape(-1,1))

pin = (device.type == "cuda")
train_loader = DataLoader(TensorDataset(Xtr, Ytr), batch_size=32, shuffle=True,
                          pin_memory=pin, num_workers=0)
val_loader   = DataLoader(TensorDataset(Xva, Yva), batch_size=128, shuffle=False,
                          pin_memory=pin, num_workers=0)
test_loader  = DataLoader(TensorDataset(Xte, Yte), batch_size=128, shuffle=False,
                          pin_memory=pin, num_workers=0)


In [18]:

n_qubits = 3
n_layers = 3  

dev = qml.device(
    "lightning.gpu",
    wires=n_qubits,
    shots=None,
    c_dtype=np.complex128,
)

def ROT_layer(w, n):
    for i in range(n): qml.Rot(*w[i], wires=i)

def U3_layer(w, n):
    for i in range(n): qml.U3(*w[i], wires=i)

def strong_entangling_layer(n):
    for i in range(n-1): qml.CNOT(wires=[i, i+1])
    qml.CNOT(wires=[n-1, 0])


In [19]:

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def qnode(inputs, weights_1, weights_2):
    qml.AngleEmbedding(features=inputs, wires=range(n_qubits), rotation='Y')
    for k in range(n_layers):
        U3_layer(weights_1[k], n_qubits)
        strong_entangling_layer(n_qubits)
        ROT_layer(weights_2[k], n_qubits)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {
    "weights_1": (n_layers, n_qubits, 3),
    "weights_2": (n_layers, n_qubits, 3),
}

qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


In [20]:

class VQNNModel(nn.Module):
    def __init__(self, nq, two_branches=True):
        super().__init__()
        self.q = qlayer
        self.two_branches = two_branches
        in_dim = 2*nq if two_branches else nq
        self.fc = nn.Linear(in_dim, 1)

    def forward(self, x):
        x1 = self.q(x)                 # (B,3)
        if self.two_branches:
            x2 = self.q(x**2)          # ikinci QNode çağrısı → maliyet 2x
            x  = torch.cat([x1, x2], dim=1)  # (B,6)
        else:
            x = x1                      # (B,3)
        x = F.relu(x)
        return self.fc(x)

model = VQNNModel(n_qubits, two_branches=True).to(device)
print(model)


VQNNModel(
  (q): <Quantum Torch Layer: func=qnode>
  (fc): Linear(in_features=6, out_features=1, bias=True)
)


In [21]:
import time

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn   = nn.L1Loss()  # MAE
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5, min_lr=1e-5
)

def run_epoch(loader, train=True):
    if train:
        model.train(True)
    else:
        model.train(False)

    total, count = 0.0, 0
    if train:
        for xb, yb in loader:
            if device.type == "cuda":
                xb = xb.to(device, non_blocking=True)
                yb = yb.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            preds = model(xb)
            loss  = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
            total += loss.item() * xb.size(0)
            count += xb.size(0)
    else:
        with torch.no_grad():
            for xb, yb in loader:
                if device.type == "cuda":
                    xb = xb.to(device, non_blocking=True)
                    yb = yb.to(device, non_blocking=True)
                preds = model(xb)
                loss  = loss_fn(preds, yb)
                total += loss.item() * xb.size(0)
                count += xb.size(0)
    return total / max(count, 1)

EPOCHS   = 100
PATIENCE = 10
best_val = float("inf")
best_ep  = -1
no_impr  = 0
best_state = None

for ep in range(1, EPOCHS + 1):
    start_t = time.time()

    tr = run_epoch(train_loader, train=True)
    va = run_epoch(val_loader,   train=False)

    scheduler.step(va)
    last_lr = scheduler._last_lr[0] if hasattr(scheduler, "_last_lr") else optimizer.param_groups[0]["lr"]

    # en iyi model kaydı
    if va < best_val - 1e-9:
        best_val  = va
        best_ep   = ep
        no_impr   = 0
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
    else:
        no_impr += 1

    elapsed = time.time() - start_t

    if device.type == "cuda":
        torch.cuda.synchronize()
        mem_mb = torch.cuda.memory_allocated() / (1024**2)
        print(f"[Epoch {ep:03d}] Train MAE={tr:.6f} | Val MAE={va:.6f} "
              f"| Best={best_val:.6f}@{best_ep:03d} | Time={elapsed:.2f}s "
              f"| GPU~{mem_mb:.1f}MB | LR={last_lr:.2e}")
    else:
        print(f"[Epoch {ep:03d}] Train MAE={tr:.6f} | Val MAE={va:.6f} "
              f"| Best={best_val:.6f}@{best_ep:03d} | Time={elapsed:.2f}s "
              f"| LR={last_lr:.2e}")

    if no_impr >= PATIENCE:
        print(f"⛳ Early stopping: {PATIENCE} epoch iyileşme yok (son en iyi {best_ep}. epoch).")
        break

if best_state is not None:
    model.load_state_dict(best_state)
    model.to(device).eval()
    print(f"✓ En iyi val MAE = {best_val:.6f} @ epoch {best_ep}")
else:
    print("Uyarı: hiç iyileşme kaydedilmedi.")


[Epoch 001] Train MAE=3.323119 | Val MAE=4.293767 | Best=4.293767@001 | Time=540.98s | GPU~16.3MB | LR=1.00e-03
[Epoch 002] Train MAE=3.235820 | Val MAE=4.178942 | Best=4.178942@002 | Time=565.68s | GPU~16.3MB | LR=1.00e-03
[Epoch 003] Train MAE=3.092115 | Val MAE=3.996053 | Best=3.996053@003 | Time=538.02s | GPU~16.3MB | LR=1.00e-03
[Epoch 004] Train MAE=2.916706 | Val MAE=3.792280 | Best=3.792280@004 | Time=538.39s | GPU~16.3MB | LR=1.00e-03
[Epoch 005] Train MAE=2.692304 | Val MAE=3.551418 | Best=3.551418@005 | Time=569.06s | GPU~16.3MB | LR=1.00e-03
[Epoch 006] Train MAE=2.495499 | Val MAE=3.375116 | Best=3.375116@006 | Time=542.30s | GPU~16.3MB | LR=1.00e-03
[Epoch 007] Train MAE=2.355198 | Val MAE=3.220464 | Best=3.220464@007 | Time=541.00s | GPU~16.3MB | LR=1.00e-03
[Epoch 008] Train MAE=2.246399 | Val MAE=3.091517 | Best=3.091517@008 | Time=570.50s | GPU~16.3MB | LR=1.00e-03
[Epoch 009] Train MAE=2.157647 | Val MAE=2.977231 | Best=2.977231@009 | Time=548.95s | GPU~16.3MB | LR=1

KeyboardInterrupt: 

In [22]:
BEST_WEIGHTS = "best_angmodel_weights.pth"

model.eval()

torch.save(model.state_dict(), BEST_WEIGHTS)
print(f"✓ Kaydedildi: {BEST_WEIGHTS}")

y_pred, y_true = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        if device.type == "cuda":
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
        yhat = model(xb).cpu().numpy().ravel()
        y_pred.append(yhat)
        y_true.append(yb.cpu().numpy().ravel())

y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

print("Test MAE:", metrics.mean_absolute_error(y_true, y_pred))
print("Test MSE:", metrics.mean_squared_error(y_true, y_pred))
print("Test R^2:",  metrics.r2_score(y_true, y_pred))


✓ Kaydedildi: best_angmodel_weights.pth
Test MAE: 1.7569183587209345
Test MSE: 7.018200152145982
Test R^2: 0.4690826875560583


In [23]:

assert best_state is not None, "best_state boş; eğitim boyunca hiç iyileşme olmamış olabilir."

model.load_state_dict(best_state)   # en iyi epoch'taki ağırlıkları RAM'e al
model.eval()
torch.save(model.state_dict(), "best_angmodel_weights.pth")
print("✓ Gerçek en iyi ağırlıklar kaydedildi: best_angmodel_weights.pth")


✓ Gerçek en iyi ağırlıklar kaydedildi: best_angmodel_weights.pth


In [24]:

y_pred, y_true = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        if device.type == "cuda":
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
        y_pred.append(model(xb).cpu().numpy().ravel())
        y_true.append(yb.cpu().numpy().ravel())

import numpy as np
from sklearn import metrics
y_pred = np.concatenate(y_pred); y_true = np.concatenate(y_true)

print("Best Test MAE:", metrics.mean_absolute_error(y_true, y_pred))
print("Best Test MSE:", metrics.mean_squared_error(y_true, y_pred))
print("Best Test R^2:",  metrics.r2_score(y_true, y_pred))


Best Test MAE: 1.7572272552391448
Best Test MSE: 7.015986575060025
Best Test R^2: 0.46925014165709367


In [30]:
try:
    feature_cols = FEATURES_USED  
except NameError:
    feature_cols = FEATURES       

print("Kullanılan kolonlar:", feature_cols)

sample = {
    "BW":   100.0,
    "COMED":1.0,
    "DOSE": 1.0,
    "TIME": 480.0,
    "EVID": 0.0,
    "MDV":  0.0,
    "AMT":  0.0,
    "CMT":  2.0,
    "DVID": 1.0,
}

sample_df = pd.DataFrame([sample], columns=feature_cols).astype(np.float64)


eps_local = 1e-8 if 'eps' not in globals() else float(eps)
x = sample_df.to_numpy(dtype=np.float64)                           
x_std = (x - mean_.astype(np.float64)) / (std_.astype(np.float64) + eps_local)  

x_t = torch.from_numpy(x_std)
if device.type == "cuda":
    x_t = x_t.to(device, non_blocking=True)


try:
    assert x_t.shape[1] == n_qubits, f"Giriş boyutu {x_t.shape[1]} != n_qubits={n_qubits}"
except NameError:
    pass  
    
model.eval()
with torch.no_grad():
    y_hat = model(x_t).item()

print(f"Tahmin DV: {y_hat:.6f}")

Kullanılan kolonlar: ['DVID', 'CMT', 'EVID']
Tahmin DV: 1.180330


In [31]:
try:
    feature_cols = FEATURES_USED 
except NameError:
    feature_cols = FEATURES      

print("Kullanılan kolonlar:", feature_cols)


sample = {
    "BW":   100.0,
    "COMED":1.0,
    "DOSE": 1.0,
    "TIME": 480.0,
    "EVID": 0.0,
    "MDV":  0.0,
    "AMT":  0.0,
    "CMT":  3.0,
    "DVID": 2.0,
}

sample_df = pd.DataFrame([sample], columns=feature_cols).astype(np.float64)

eps_local = 1e-8 if 'eps' not in globals() else float(eps)
x = sample_df.to_numpy(dtype=np.float64)                           
x_std = (x - mean_.astype(np.float64)) / (std_.astype(np.float64) + eps_local)

x_t = torch.from_numpy(x_std)
if device.type == "cuda":
    x_t = x_t.to(device, non_blocking=True)


try:
    assert x_t.shape[1] == n_qubits, f"Giriş boyutu {x_t.shape[1]} != n_qubits={n_qubits}"
except NameError:
    pass  
model.eval()
with torch.no_grad():
    y_hat = model(x_t).item()

print(f"Tahmin DV: {y_hat:.6f}")

Kullanılan kolonlar: ['DVID', 'CMT', 'EVID']
Tahmin DV: 6.524476
